### Import Libraries

In [1]:
import pandas as pd # Dataset operations
import numpy as np # math and array operations
import scipy.linalg as la

### Import Dataset
Import dataset with the following columns:
- Propierties
- Units
- Max or Min Criteria
- Next columns are for each material with its properties values

In [2]:
df = pd.read_excel("Materiales.xlsx")
df.head()

,Propiedad,Unidades,Criterio,PP,LDPE,HDPE,N6
0,Esfuerzo fluencia,Mpa,Max,41.40,78.000,38.00,79.00
1,Módulo de elasticidad,Gpa,Max,1.77,0.380,1.50,2.90
2,Densidad,g/cm^3,Min,0.92,0.925,0.96,1.14
3,Absorción de agua,%,Min,0.02,0.015,0.20,1.80
4,Costo de la materia prima,USD/kg,Min,0.97,1.060,0.90,2.10


In [3]:
# List of criterias
criterias = df["Propiedad"].tolist()
criterias

['Esfuerzo fluencia',
 'Módulo de elasticidad',
 'Densidad',
 'Absorción de agua',
 'Costo de la materia prima']

In [4]:
# List of alternatives
alternatives =  df.columns.tolist()[3:]
alternatives

['PP', 'LDPE', 'HDPE', 'N6']

# AHP

### Paired Matrix

In [5]:
def paired_matrix(names):
    size = len(names) # Length of the list of criterias/alternatives
    M = np.zeros((size,size),dtype=object) # Zeros matrix to save the paired comparisons
    cont = 1 # counter to compare each pair of criterias/alternatives once
    
    for i in range(size):
        for j in range(cont,size):
            M[i,j] = str(input("Paired comparsion between {} & {}: ".format(names[i],names[j])))
           
            if "/" in M[i,j]: # Numbers input as fraction
                num, den = M[i,j].split("/")
                M[i,j] = int(num)/int(den)
                M[j,i] = int(den)/int(num)
            else:
                M[i,j] = float(M[i,j]) # Str to float
                M[j,i] = float(1/M[i,j])
                
        cont +=1
        
    np.fill_diagonal(M, 1) # Fill diagonal with 1
    
    return M.astype(float)

### Lambda function

In [6]:
def lambda_function(matrix):
    vals, vects = la.eig(matrix) # Eigenvalues and eigenvectors
    maxcol = list(vals).index(max(vals)) # Column with max eigenvalues
    eigenvect = np.real(vects[:,maxcol]) # Getting the maxeigenvalues of the eigenvectors (Real part)
    lambda_values = eigenvect/sum(eigenvect) # Eigenvalues normalized 0-1
    return lambda_values

### Consistency criteria function

In [7]:
def consistency(matrix,lamb):
    P = matrix.sum(axis=0) # Sum matrix columns
    lambda_max = np.dot(P, lamb) # Dot product between P and Lambda values
    n = len(matrix) # Number of criterias/alternatives
    CI = (lambda_max-n)/(n-1) # Consitency index
    RI = [0, 0, 0.58, 0.89, 1.12, 1.26, 1.36, 1.41, 1.42, 1.49, 1.52, 1.54, 1.56, 1.58] # Random consitency index
    CR = CI/RI[n-1] # Consitency rate
    return CR

### Criterias Matrix

In [8]:
condition = True # Condition of consistency

while condition:
    
    m_cri = paired_matrix(criterias) # Paired matrix
    
    lambda_cri = lambda_function(m_cri) # Max eigenvalues of criterias
    
    print(" ")
    print(list(zip(criterias,lambda_cri))) # Zip criterias and eigenvalues
    
    CR = consistency(m_cri,lambda_cri) # Consistency test
    
    if CR <= 0.1:
        print(" ")
        print("The matrix is consistent with a value of {:.3f} or {:.2f}%".format(CR,CR*100))
        print(" ")
        print(m_cri)
        print(" ")
        condition = False
    else:
        print(" ")
        print("The matrix is NOT consistent with a value of {:.3f} or {:.2f}%".format(CR,CR*100))
        print("Review the paired comparison values")
        print(" ")
        print(m_cri)
        print(" ")

Paired comparsion between Esfuerzo fluencia & Módulo de elasticidad: 1/5
Paired comparsion between Esfuerzo fluencia & Densidad: 1/7
Paired comparsion between Esfuerzo fluencia & Absorción de agua: 1
Paired comparsion between Esfuerzo fluencia & Costo de la materia prima: 1/7
Paired comparsion between Módulo de elasticidad & Densidad: 1/3
Paired comparsion between Módulo de elasticidad & Absorción de agua: 5
Paired comparsion between Módulo de elasticidad & Costo de la materia prima: 1/5
Paired comparsion between Densidad & Absorción de agua: 5
Paired comparsion between Densidad & Costo de la materia prima: 1
Paired comparsion between Absorción de agua & Costo de la materia prima: 1/9
 
[('Esfuerzo fluencia', 0.04354949761432059), ('Módulo de elasticidad', 0.1512755846415514), ('Densidad', 0.33415358138113027), ('Absorción de agua', 0.044615414191505234), ('Costo de la materia prima', 0.42640592217149254)]
 
The matrix is consistent with a value of 0.048 or 4.78%
 
[[1.         0.2    

In [9]:
# Show it as a Dataframe
df_cri = pd.DataFrame(m_cri)
df_cri.insert(0, "Propiedad", criterias)
df_cri.columns = ["Propiedad"] + criterias
df_cri

,Propiedad,Esfuerzo fluencia,Módulo de elasticidad,Densidad,Absorción de agua,Costo de la materia prima
0,Esfuerzo fluencia,1.0,0.2,0.142857,1.0,0.142857
1,Módulo de elasticidad,5.0,1.0,0.333333,5.0,0.200000
2,Densidad,7.0,3.0,1.000000,5.0,1.000000
3,Absorción de agua,1.0,0.2,0.200000,1.0,0.111111
4,Costo de la materia prima,7.0,5.0,1.000000,9.0,1.000000


### Alternatives Matrices

In [10]:
m_alt = np.zeros((len(criterias),len(alternatives),len(alternatives))) # Zeros hypermatrix to save each paired comparison
lambda_alt = np.zeros((len(alternatives),len(criterias))) # Zeros matriz to save eigenvalues of alternatives

for i in range(len(criterias)):
    
    print("\033[1m {} \033[0m".format(criterias[i])) # Print name criteria
    
    condition = True # Condition of consistency
    
    while condition:
        m_alt[i] = paired_matrix(alternatives) # Paired comparison
        
        lambda_alt[:,i] = lambda_function(m_alt[i]) # Max eigenvalues of criterias
        
        print(" ")
        print(list(zip(alternatives,lambda_alt[:,i]))) # Zip altenarives and eigenvalues
              
        CR = consistency(m_alt[i],lambda_alt[:,i]) # Consistency rate
              
        if CR <= 0.1:
            print(" ")
            print("The matrix is consistent with a value of {:.3f} or {:.2f}%".format(CR,CR*100))
            print(" ")
            print(m_alt[i])
            print(" ")
            condition = False
        else:
            print(" ")
            print("The matrix is NOT consistent with a value of {:.3f} or {:.2f}%".format(CR,CR*100))
            print("Review the paired comparison values")
            print(" ")
            print(m_alt[i])
            print(" ")
        

 Esfuerzo fluencia 
Paired comparsion between PP & LDPE: 1/7
Paired comparsion between PP & HDPE: 2
Paired comparsion between PP & N6: 1/7
Paired comparsion between LDPE & HDPE: 7
Paired comparsion between LDPE & N6: 1
Paired comparsion between HDPE & N6: 1/7
 
[('PP', 0.07531373019088676), ('LDPE', 0.43584794485493544), ('HDPE', 0.05299038009924251), ('N6', 0.43584794485493544)]
 
The matrix is consistent with a value of 0.023 or 2.27%
 
[[1.         0.14285714 2.         0.14285714]
 [7.         1.         7.         1.        ]
 [0.5        0.14285714 1.         0.14285714]
 [7.         1.         7.         1.        ]]
 
 Módulo de elasticidad 
Paired comparsion between PP & LDPE: 7
Paired comparsion between PP & HDPE: 2
Paired comparsion between PP & N6: 1/5
Paired comparsion between LDPE & HDPE: 1/5
Paired comparsion between LDPE & N6: 1/9
Paired comparsion between HDPE & N6: 1/7
 
[('PP', 0.1932288612232936), ('LDPE', 0.03791034395149584), ('HDPE', 0.11654965723339464), ('N6', 

In [11]:
m_alt # Paired comparison matrices for alternatives

array([[[1.        , 0.14285714, 2.        , 0.14285714],
        [7.        , 1.        , 7.        , 1.        ],
        [0.5       , 0.14285714, 1.        , 0.14285714],
        [7.        , 1.        , 7.        , 1.        ]],

       [[1.        , 7.        , 2.        , 0.2       ],
        [0.14285714, 1.        , 0.2       , 0.11111111],
        [0.5       , 5.        , 1.        , 0.14285714],
        [5.        , 9.        , 7.        , 1.        ]],

       [[1.        , 2.        , 3.        , 7.        ],
        [0.5       , 1.        , 2.        , 5.        ],
        [0.33333333, 0.5       , 1.        , 3.        ],
        [0.14285714, 0.2       , 0.33333333, 1.        ]],

       [[1.        , 0.5       , 5.        , 9.        ],
        [2.        , 1.        , 5.        , 9.        ],
        [0.2       , 0.2       , 1.        , 5.        ],
        [0.11111111, 0.11111111, 0.2       , 1.        ]],

       [[1.        , 3.        , 0.5       , 7.        ],
      

In [12]:
lambda_alt # Max eigen values for each alternatives

array([[0.07531373, 0.19322886, 0.49053883, 0.35440335, 0.31536508],
       [0.43584794, 0.03791034, 0.28780077, 0.49831616, 0.14338655],
       [0.05299038, 0.11654966, 0.1615492 , 0.11068504, 0.49767982],
       [0.43584794, 0.65231114, 0.06011121, 0.03659545, 0.04356855]])

In [13]:
# Show it as a Dataframe
df_alt = pd.DataFrame(lambda_alt)
df_alt.insert(0, "Material", alternatives)
df_alt.columns = ["Material"] + criterias
df_alt

,Material,Esfuerzo fluencia,Módulo de elasticidad,Densidad,Absorción de agua,Costo de la materia prima
0,PP,0.075314,0.193229,0.490539,0.354403,0.315365
1,LDPE,0.435848,0.037910,0.287801,0.498316,0.143387
2,HDPE,0.052990,0.116550,0.161549,0.110685,0.497680
3,N6,0.435848,0.652311,0.060111,0.036595,0.043569


In [14]:
score = np.zeros((len(alternatives),1))
for i in range(len(alternatives)):
    score[i] = np.dot(lambda_alt[i,:], lambda_cri)
score

array([[0.34671138],
       [0.20425898],
       [0.29107295],
       [0.15795669]])

### Results

In [20]:
# Show it as a Dataframe
df_end = pd.DataFrame(score)
df_end.insert(0, "Material", alternatives)
df_end.columns = ["Material","Score"]
df_end

,Material,Score
0,PP,0.346711
1,LDPE,0.204259
2,HDPE,0.291073
3,N6,0.157957


In [21]:
df_end = df_end.sort_values(by=["Score"], ascending = False)
df_end.insert(0, "Ranking", list(range(1,len(alternatives)+1)))
df_end

,Ranking,Material,Score
0,1,PP,0.346711
2,2,HDPE,0.291073
1,3,LDPE,0.204259
3,4,N6,0.157957
